**Goal:** Learn foundation GIS and create an dinteractive useful map with basic gis python and Fused Mapbox

1. Load Data
2. Geocode
3. Map stations
4. HeatMap by Dencity
5. Flood 

In [8]:
#load MTA data with stops location 

import pandas as pd

# Read the text file (adjust separator as needed)
df = pd.read_csv("stops.txt", delimiter=",")  # Change delimiter based on your file format

df.head(3)

<ipython-input-8-20c2c5ec3418>:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
0,101,Van Cortlandt Park-242 St,40.889248,-73.898583,1.0,NaN
1,101N,Van Cortlandt Park-242 St,40.889248,-73.898583,NaN,101
2,101S,Van Cortlandt Park-242 St,40.889248,-73.898583,NaN,101


In [9]:
# Extract unique stop names
unique_stop_names = df['stop_name'].unique()

# Print the unique stop names
#unique_stop_names

In [10]:
# Install folium if it's not already installed
import micropip

# Install folium
await micropip.install("folium")

# Import folium after installation
import folium




# Assuming your DataFrame 'df' has 'stop_lat', 'stop_lon', and 'stop_name' columns
# Create a map centered on New York City
m = folium.Map(location=[40.7128, -74.0060], tiles='openstreetmap', zoom_start=12)

# Add smaller dots (CircleMarkers) to the map
for idx, row in df.iterrows():
    folium.CircleMarker(
        location=[row['stop_lat'], row['stop_lon']],
        radius=3,  # Radius controls the size of the dot (adjust as needed)
        color="blue",  # Outline color of the circle
        fill=True,
        fill_color="blue",  # Fill color of the circle
        fill_opacity=0.5,
        popup=row['stop_name']  # Popup when you click on the dot
    ).add_to(m)

# Display the map
m


In [12]:
#load population dataset
import geopandas as gpd
df_population = gpd.read_file('New_York_City_Population_By_Community_Districts_20241103.csv')
df_population.head(3)

,Borough,CD Number,CD Name,1970 Population,1980 Population,1990 Population,2000 Population,2010 Population,geometry
0,Bronx,1,"Melrose, Mott Haven, Port Morris",138557,78441,77214,82159,91497,None
1,Bronx,2,"Hunts Point, Longwood",99493,34399,39443,46824,52246,None
2,Bronx,3,"Morrisania, Crotona Park East",150636,53635,57162,68574,79762,None


In [14]:
import pandas as pd
import requests
import time

# Define the geocoding function
def geocode_address(api_key, address):
    complete_address = f"{address}, New York, NY"  # Adjust based on your dataset context
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={complete_address}&key={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        


# API Key
api_key = ***** # Replace with your actual API key

# List to hold failed addresses
failed_addresses = []

# Apply the geocoding function to the DataFrame
df_population[['Latitude', 'Longitude']] = df_population['CD Name'].apply(
    lambda x: pd.Series(geocode_address(api_key, x) if (result := geocode_address(api_key, x)) else failed_addresses.append(x))
)

df_population.head(3)
df_population.to_csv('df_population_lat_long.csv', index=False)

In [19]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import numpy as np # import numpy



# Create a base map centered on the specified location
m_5 = folium.Map(location=[40.7128, -74.0060], tiles='cartodb positron', zoom_start=12)

# Ensure all values used in heat_data are numeric
df_population['Latitude'] = pd.to_numeric(df_population['Latitude'], errors='coerce')
df_population['Longitude'] = pd.to_numeric(df_population['Longitude'], errors='coerce')
df_population['2010 Population'] = pd.to_numeric(df_population['2010 Population'], errors='coerce')

# Prepare data for the heatmap by multiplying the population by the latitude/longitude pair
# This helps the heatmap represent the population density effectively
heat_data = [[row['Latitude'], row['Longitude'], row['2010 Population']] for index, row in df_population.iterrows() if not np.isnan(row['Latitude']) and not np.isnan(row['Longitude']) and not np.isnan(row['2010 Population'])] # Filter out rows with NaN values

# Add a heatmap layer to the map
HeatMap(heat_data, max_val=df_population['2010 Population'].max()).add_to(m_5)

# Display the map
m_5

<ipython-input-19-a8a46c06f885>:21: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(heat_data, max_val=df_population['2010 Population'].max()).add_to(m_5)


In [25]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import numpy as np

# Sample data structure
# df_population should contain 'Latitude', 'Longitude', '2010 Population', and 'City' columns
# Example:
# df_population = pd.DataFrame({
#     'Latitude': [40.7128, 40.73, 40.74],
#     'Longitude': [-74.0060, -74.01, -74.02],
#     '2010 Population': [5000, 2000, 3000],
#     'City': ['New York', 'Brooklyn', 'Queens']
# })

# Create a base map centered on New York City
m_5 = folium.Map(location=[40.7128, -74.0060], tiles='cartodb positron', zoom_start=12)

# Ensure all values used in heat_data are numeric
df_population['Latitude'] = pd.to_numeric(df_population['Latitude'], errors='coerce')
df_population['Longitude'] = pd.to_numeric(df_population['Longitude'], errors='coerce')
df_population['2010 Population'] = pd.to_numeric(df_population['2010 Population'], errors='coerce')

# Filter data and prepare heat_data
heat_data = [
    [row['Latitude'], row['Longitude'], row['2010 Population']]
    for index, row in df_population.iterrows()
    if not np.isnan(row['Latitude']) and not np.isnan(row['Longitude']) and not np.isnan(row['2010 Population'])
]

# Add HeatMap layer with adjusted radius and blur for a more unified area effect
HeatMap(heat_data, radius=30, blur=30).add_to(m_5)

# Add markers with tooltips for each location
for _, row in df_population.iterrows():
    if not np.isnan(row['Latitude']) and not np.isnan(row['Longitude']):
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            tooltip=row['CD Name']  # Display the city name on hover
        ).add_to(m_5)

# Display the map
m_5


In [29]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import numpy as np


# Load the MTA stop data
df_mta = pd.read_csv("stops.txt")  # Make sure stops.txt contains 'stop_lat', 'stop_lon', and 'stop_name' columns

# Create a base map centered on New York City
m = folium.Map(location=[40.7128, -74.0060], tiles='cartodb positron', zoom_start=12)


# Prepare heatmap data
heat_data = [
    [row['Latitude'], row['Longitude'], row['2010 Population']]
    for _, row in df_population.iterrows()
    if not np.isnan(row['Latitude']) and not np.isnan(row['Longitude']) and not np.isnan(row['2010 Population'])
]

# Add HeatMap layer with adjusted radius and blur
HeatMap(heat_data, radius=30, blur=30).add_to(m)

# Add city markers with tooltips
for _, row in df_population.iterrows():
    if not np.isnan(row['Latitude']) and not np.isnan(row['Longitude']):
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            tooltip=row['CD Name']  # Display the city name on hover
        ).add_to(m)

# Add CircleMarkers for each MTA stop with popup for the stop name
for idx, row in df_mta.iterrows():
    folium.CircleMarker(
        location=[row['stop_lat'], row['stop_lon']],
        radius=3,  # Small radius for the MTA stops
        color="blue",  # Outline color of the circle
        fill=True,
        fill_color="blue",
        fill_opacity=0.5,
        popup=row['stop_name']  # Popup with stop name on click
    ).add_to(m)

# Display the map
m
